In [2]:
from backtesting.test import SMA
import pandas as pd
from finVADER import finvader
import matplotlib.pyplot as plt
import numpy as np

C:\Users\DELL\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\backtesting\test\__init__.py:8: FutureWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  return pd.read_csv(join(dirname(__file__), filename),
C:\Users\DELL\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\backtesting\test\__init__.py:8: FutureWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  return pd.read_csv(join(dirname(__file__), filename),
[nltk_data] Downlo

In [3]:
from backtesting import Backtest
from backtesting import Strategy
from backtesting.lib import crossover


class SmaCrosss(Strategy):
    # Define the two MA lags as *class variables*
    # for later optimization
    n1 = 2
    n2 = 5
    
    def init(self):
        # Precompute the two moving averages
        self.sma1 = self.I(SMA, self.data.Close, self.n1)
        self.sma2 = self.I(SMA, self.data.Close, self.n2)
    
    def next(self):
        index = len(self.data) - 1
        # If sma1 crosses above sma2, close any existing
        # short trades, and buy the asset
        if crossover(self.sma1, self.sma2):
            self.position.close()
            self.buy()

        # Else, if sma1 crosses below sma2, close any existing
        # long trades, and sell the asset
        elif crossover(self.sma2, self.sma1):
            self.position.close()
            self.sell()

data_tesla = pd.read_csv('data_tsla.csv', index_col='Datetime', parse_dates=True)
bt = Backtest(data_tesla, SmaCrosss, cash= 10000, commission=.002)

stat = bt.run()
bt.plot()
stat

C:\Users\DELL\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\backtesting\_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_drag' property; using the latest value
  fig = gridplot(
C:\Users\DELL\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\backtesting\_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_scroll' property; using the latest value
  fig = gridplot(


Start                     2024-01-02 09:30:00
End                       2024-01-26 15:30:00
Duration                     24 days 06:00:00
Exposure Time [%]                   97.435897
Equity Final [$]                 10417.860007
Equity Peak [$]                  10458.268195
Return [%]                             4.1786
Buy & Hold Return [%]              -26.352766
Return (Ann.) [%]                  101.805145
Volatility (Ann.) [%]              114.130894
Sharpe Ratio                         0.892003
Sortino Ratio                        4.550859
Calmar Ratio                            7.367
Max. Drawdown [%]                  -13.819077
Avg. Drawdown [%]                   -2.656304
Max. Drawdown Duration       22 days 02:00:00
Avg. Drawdown Duration        3 days 10:30:00
# Trades                                   48
Win Rate [%]                        35.416667
Best Trade [%]                      13.108357
Worst Trade [%]                     -2.217239
Avg. Trade [%]                    

In [18]:
class finVADERnSMA(Strategy):
    threshold = 0.33
    sma1 = 2
    sma2 = 5
    data_cmt = pd.read_csv('tsla_cleaned.csv')

    def init(self):
        self.sma1 = self.I(SMA, self.data.Close, self.sma1)
        self.sma2 = self.I(SMA, self.data.Close, self.sma2)
        super().init()

    def next(self):
        #if crossover(self.sma1, self.sma2):
        #    self.buy()
        #elif crossover(self.sma2, self.sma1):
        #    self.sell()
        #index = len(self.data) - 1
        # # If sma1 crosses above sma2, close any existing
        # # short trades, and buy the asset
        if crossover(self.sma1, self.sma2):
        #    self.position.close()
            self.buy()

        # Else, if sma1 crosses below sma2, close any existing
        # long trades, and sell the asset
        elif crossover(self.sma2, self.sma1):
             #self.position.close()
            self.sell()
        super().next()
        
        
    
    def compute_sma(self, data):
        data['sma1'] = data['Close'].rolling(self.sma1).mean()
        data['sma2'] = data['Close'].rolling(self.sma2).mean()
        pass

    def compute_finVADER(self, data_cmt):
        data_cmt['date'] = pd.to_datetime(data_cmt['date'])
        data_cmt.sort_values('date', inplace=True)
        data_cmt['no_sw_cmt'] = data_cmt['no_sw_cmt'].astype(str)
        data_cmt['compound'] = data_cmt['no_sw_cmt'].apply(finvader,use_sentibignomics = True, use_henry = True, indicator="compound")
        data_cmt = data_cmt[data_cmt['compound'] != 0]
        data_cmt = data_cmt.groupby('date')['compound'].agg(['min', 'max']).reset_index()
        data_cmt['final_score'] = data_cmt['min'] + data_cmt['max']
        data_cmt['signal'] = data_cmt['final_score'].apply(lambda x: 'buy' if x > self.threshold else 'sell')
        return data_cmt

    def generate_signals(self, data, data_cmt):
        self.compute_sma(data)
        data_cmt = self.compute_finVADER(data_cmt)
        data = data.merge(data_cmt[['date', 'signal']], how='left', on='date')
        data['signal'].fillna(method='ffill', inplace=True)
        data['final_signal'] = data.apply(lambda x: 'buy' if x['sma1'] > x['sma2'] 
                                                else 'sell' if x['sma1'] < x['sma2'] 
                                                else x['signal'], axis=1)
        return data
    

In [19]:
#cmts = pd.read_csv('tsla_cleaned.csv')
stock = pd.read_csv('data_tsla.csv', index_col='Datetime', parse_dates=True)
#stock = stock.rename(columns={'Datetime': 'date'})
#cmts['date'] = pd.to_datetime(cmts['date'])
#stock['date'] = pd.to_datetime(stock['Datetime'])
# Khởi tạo một instance của finVADERnSMA
#strategy = finVADERnSMA(sma1=2, sma2=5, threshold=0.33)

# # Tính toán và lấy tín hiệu
# signals = strategy.generate_signals(stock, cmts)

# # Tạo một DataFrame mới kết hợp dữ liệu giá cổ phiếu và tín hiệu
# data_with_signals = pd.concat([data_tesla, signals], axis=1)

# Sử dụng Backtest với dữ liệu mới
bt1 = Backtest(stock, finVADERnSMA, cash=10000, commission=.002)
stat = bt1.run()
bt1.plot()
stat

C:\Users\DELL\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\backtesting\_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_drag' property; using the latest value
  fig = gridplot(
C:\Users\DELL\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\backtesting\_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_scroll' property; using the latest value
  fig = gridplot(


Start                     2024-01-02 09:30:00
End                       2024-01-26 15:30:00
Duration                     24 days 06:00:00
Exposure Time [%]                   97.435897
Equity Final [$]                  13327.41761
Equity Peak [$]                   13383.39709
Return [%]                          33.274176
Buy & Hold Return [%]              -26.352766
Return (Ann.) [%]                 5649.538576
Volatility (Ann.) [%]             2850.292665
Sharpe Ratio                         1.982091
Sortino Ratio                     1055.870398
Calmar Ratio                      1433.923036
Max. Drawdown [%]                   -3.939918
Avg. Drawdown [%]                   -1.157016
Max. Drawdown Duration        4 days 19:00:00
Avg. Drawdown Duration        1 days 02:19:00
# Trades                                   17
Win Rate [%]                            100.0
Best Trade [%]                      26.316807
Worst Trade [%]                      0.890981
Avg. Trade [%]                    

In [14]:
class finVADERnSMA1(Strategy):
    threshold = 0.33
    sma1_period = 2
    sma2_period = 5
    data_cmt = pd.read_csv('tsla_cleaned.csv')

    def init(self):
        self.sma1 = self.I(SMA, self.data.Close, self.sma1_period)
        self.sma2 = self.I(SMA, self.data.Close, self.sma2_period)
        super().init()

    def compute_finVADER(self, data_cmt):
        data_cmt['date'] = pd.to_datetime(data_cmt['date'])
        data_cmt.sort_values('date', inplace=True)
        data_cmt['no_sw_cmt'] = data_cmt['no_sw_cmt'].astype(str)
        data_cmt['compound'] = data_cmt['no_sw_cmt'].apply(finvader, use_sentibignomics=True, use_henry=True, indicator="compound")
        data_cmt = data_cmt[data_cmt['compound'] != 0]
        data_cmt = data_cmt.groupby('date')['compound'].agg(['min', 'max']).reset_index()
        data_cmt['final_score'] = data_cmt['min'] + data_cmt['max']
        data_cmt['signal'] = np.where(data_cmt['final_score'] > self.threshold, 'buy', 'sell')
        return data_cmt

    def generate_signals(self, data, data_cmt):
        # Sử dụng giá trị SMA đã tính từ init
        data['final_signal'] = np.where(data['sma1'] > data['sma2'], 'buy',
                                        np.where(data['sma1'] < data['sma2'], 'sell', np.nan))
          # Tính chỉ báo FinVADER và gộp với tín hiệu SMA
        data_cmt = self.compute_finVADER(data_cmt)
        data['date'] = data.index
        data['date'] = pd.to_datetime(data['date'])
        #data = data.merge(data_cmt[['date', 'signal']], how='left', on='date')
        #data['final_signal'] = data['signal'].fillna(data['final_signal'])
        # Hợp nhất tín hiệu từ cả SMA và FinVADER
        data['final_signal'] = np.where(data['final_signal'].notna(), data['final_signal'], data_cmt['signal'])
        #data['final_signal'] = np.where(data['signal'].notna(), data['signal'], data['final_signal'])
        
        return data
        
    def next(self, data, data_cmt):
        # Compute SMA signals
        self.compute_sma(self.data)
    
        # Compute finVADER signals
        data_cmt = self.compute_finVADER(self.data_cmt)
    
        # Merge the dataframes
        self.data = self.data.merge(data_cmt[['date', 'signal']], how='left', on='date')
        self.data['signal'].fillna(method='ffill', inplace=True)
    
        # Generate final signal
        self.data['final_signal'] = self.data.apply(lambda x: 'buy' if x['sma1'] > x['sma2'] 
                                                else 'sell' if x['sma1'] < x['sma2'] 
                                                else x['signal'], axis=1)
    
        # Perform buy or sell based on the final signal
        if self.data['final_signal'].iloc[-1] == 'buy':
            self.buy()
        elif self.data['final_signal'].iloc[-1] == 'sell':
            self.sell()
        super().next()


In [15]:
stock = pd.read_csv('data_tsla.csv', index_col='Datetime', parse_dates=True)
#stock = stock.rename(columns={'Datetime': 'date'})

# Sử dụng Backtest với dữ liệu mới
bt2 = Backtest(stock, finVADERnSMA1, cash=10000, commission=.002)
stat2 = bt2.run()
bt2.plot()
stat2

TypeError: finVADERnSMA1.next() missing 2 required positional arguments: 'data' and 'data_cmt'